In [1]:
import icartt
import pandas as pd
import numpy as np
import datetime
from math import pi
import os

In [65]:
#import csv files with information
file_info_file = "C:/Users/cphal/OneDrive/Desktop/Aerosols/Module A/Metadata/module_a_bins.csv"
binlocations = pd.read_csv(file_info_file)

In [66]:
#function purpose: get new file name
def get_new_name(filename, binlocations):
    #filename is uppercase
    filename = filename.upper()

    #find associated row by looping through files
    campaign = ''
    instrument = ''
    i = 0
    campaign_found = False
    instrument_found = False
    while (i < len(binlocations) or (not campaign_found and not instrument_found)):
        if (not campaign_found and isinstance(binlocations.iloc[i]['Campaign'],str)):
            if(binlocations.iloc[i]['Campaign'] in filename):
                campaign = binlocations.iloc[i]['Campaign']
                campaign_found = True
        if (not instrument_found and isinstance(binlocations.iloc[i]['Filename/Instrument'],str)):
            if(binlocations.iloc[i]['Filename/Instrument'] in filename):
                instrument = binlocations.iloc[i]['Filename nickname']
                instrument_found = True
        i += 1

    if(not campaign_found or not instrument_found):
        print('FATAL ERROR: campaign/instrument not found for ' + filename)
        exit(0)
    
    #find new filename and get its index
    new_file_name = campaign + '_' + instrument + '_'

    return new_file_name

In [75]:
#create diameter dataframe
def createDiamDf():
    diameter_file = "C:/Users/cphal/OneDrive/Desktop/Aerosols/Module A/NASA diameters.csv"

    #diameters_info = pd.read_csv(diameter_file, delimiter=',', encoding='utf-8')
    file = open(diameter_file,'r')
    count = 0
    dfDiam = pd.DataFrame(columns = ['file_name', 'row', 'column_start', 'column_end', 'diameters'], index = range(0,23))

    #cycle through file, extract values, and put them where they're supposed to be
    for line in file:
        if count > 0:
            line.strip('/n')
            nextline = line.split('[')
            nextline[1] = '[' + nextline[1]
            firstpart = nextline[0].split(',')

            dfDiam.at[count-1, 'file_name'] = firstpart[0]
            dfDiam.at[count-1, 'row'] = firstpart[1]
            dfDiam.at[count-1, 'column_start'] = firstpart[2]
            dfDiam.at[count-1, 'column_end'] = firstpart[3]
            dfDiam.at[count-1, 'diameters'] = nextline[1]
        count += 1

    return dfDiam

In [14]:
#function purpose: find nearest value (d_N1, d_N2, d_V1, d_V2) in numpy array (instrument measured diameters)
def find_nearest(array, value):
    n = [abs(i-value) for i in array]
    idx = n.index(min(n))
    return idx

In [81]:
#if not looping
filebeginning = 'C:/Users/cphal/OneDrive/Desktop/Aerosols/Module A/Module_A_Failure/'
filename = filebeginning + 'FIREXAQ-LARGE-SMPS_DC8_20190716_R0.ict'

In [80]:
#module a code
def module_a(filename, dfDiam):
        
    new_file_name = get_new_name(filename, binlocations)

    #create dfDiam
    ind = dfDiam.index[dfDiam['file_name'] == new_file_name].tolist()
    print(ind)

    #make sure not empty
    if (len(ind) > 1):
        print('FATAL ERROR: more than 1 match for ' + new_file_name)
        exit(0)

    #format diameter input properly
    diameters = dfDiam.at[ind[0],'diameters']
    diameters = diameters.strip('][\n').split(',')
    diameters = [float(x) for x in diameters]
    size_dist_diameter_input = pd.Series(diameters)

    #load in ict file to python
    ict = icartt.Dataset(filename)
    df = ict.data[:]

    df = pd.DataFrame(df)

    varnames = [x for x in ict.variables]

    column_start = int(dfDiam.at[ind[0],'column_start'])
    column_end = int(dfDiam.at[ind[0],'column_end'])

    varnames = [x for x in ict.variables]
    binnames = varnames[column_start:column_end]

    size_dist_input_original = df[binnames]

    #transpose size_dist_input to get it to work with Module A
    size_dist_input = pd.DataFrame.transpose(size_dist_input_original)

    #create new vector to hold column names
    ser=pd.Series(range(np.size(size_dist_input,1)))
    #assign this to be the columns
    size_dist_input.rename(columns = ser, inplace = True)

    #replace all input values less than 0 with np.nan
    #FIXME: consider LLOD of instrument instead
    size_dist_input.mask(size_dist_input <= 0, np.nan , inplace=True )

    #create list to hold input values
    d_Nx_list = [300, 1000]
    d_Vx_list = [500,1500]
    bin_num = 3
    #get_inputs(d_Nx_list, d_Vx_list)
    #3 bins , d_Nx = [10.5, 30] , d_Vx = [25,40]

    #reformat date and time
    date_time = ict.times
    datetimedf = pd.DataFrame(date_time, columns= ['datetime'])
    date = str(date_time[0])
    date = datetime.datetime.strptime(date, '%Y-%m-%dT%H:%M:%S.%f000').strftime('%Y_%m_%d')

    #make sure properly formatted
    size_dist_diameter_input.reset_index(drop=True)
    size_dist_diameter_input = size_dist_diameter_input.astype(float)

    #calculate dVdlogdP using dNdlogdP
    diameter_power = np.power(size_dist_diameter_input, 3)
    diameter_power = diameter_power.to_numpy()
    size_dist_volume = pi/6*size_dist_input.mul(diameter_power,axis='index')

    #calculate dVdlogdP using dNdlogdP TEST PURPOSES ONLY
    #size_dist_volume_slow = pd.DataFrame().reindex_like(size_dist_input)
    #for m in range(np.size(size_dist_volume_slow,1)):
    #for n in range(len(size_dist_diameter_input)):
    #size_dist_volume_slow.values[n,m]=pi/6* np.power(size_dist_diameter_input, 3).values[n]*size_dist_input.values[n,m]

    #prepare for trapz calculation
    size_dist_input[np.isnan(size_dist_input)] = 0 
    size_dist_volume[np.isnan(size_dist_volume)] = 0 

    ln_size_dist_diameter_input=np.log(size_dist_diameter_input)

    ##MODIFIED CODE
    #instead of having [num_bins] separate arrays with equal length, have [num_bins] columns in a dataframe
    #have a row for each data type needed to store an array in
    #multidimensional array! TODO: draw a picture of this

    #find nearest index in size_dist_diameter_input
    d_Nx_nearest = []
    d_Vx_nearest = []

    #for num_bins, find nearest index in size_dist_diameter input
    for i in range(len(d_Nx_list)):
        x = find_nearest(size_dist_diameter_input, d_Nx_list[i])
        d_Nx_nearest.append(x)

    for i in range(len(d_Vx_list)):
        x = find_nearest(size_dist_diameter_input, d_Vx_list[i])
        d_Vx_nearest.append(x)

    #create empty dataframes to hold num_bins: should be (num_bins) x (num_columns in size_dist)
    num_bins = len(d_Nx_list) + 1

    N_df = pd.DataFrame(index = range(np.size(size_dist_input,1)), columns = range(num_bins))
    F_N_df = pd.DataFrame(index = range(np.size(size_dist_input,1)), columns = range(num_bins))
    V_df = pd.DataFrame(index = range(np.size(size_dist_input,1)), columns = range(num_bins))
    F_V_df = pd.DataFrame(index = range(np.size(size_dist_input,1)), columns = range(num_bins))

    #add beginning and ending indexes for the divisions
    d_Nx_nearest.insert(0,0)
    d_Nx_nearest.append(len(size_dist_diameter_input))

    d_Vx_nearest.insert(0,0)
    d_Vx_nearest.append(len(size_dist_diameter_input))

    #for each bin, make a smaller dataframe with the diameter bins desired. Then apply trapz calculation by row
    #and add result to dataframe in adjacent bin
    for i in range(num_bins):
        if i == 0: 
            small_df = size_dist_input.iloc[0:(d_Nx_nearest[i+1]),:]
            N_df.iloc[:,i] = np.trapz(small_df, x=ln_size_dist_diameter_input[0:(d_Nx_nearest[i+1])], axis=0)
        elif i > 0:
            small_df = size_dist_input.iloc[(d_Nx_nearest[i]+1):(d_Nx_nearest[i+1]),:]
            N_df.iloc[:,i] = np.trapz(small_df, x=ln_size_dist_diameter_input[(d_Nx_nearest[i]+1):(d_Nx_nearest[i+1])], axis=0)

    for i in range(num_bins):
        if i == 0: 
            small_df = size_dist_volume.iloc[0:(d_Vx_nearest[i+1]),:]
            V_df.iloc[:,i] = np.trapz(small_df, x=ln_size_dist_diameter_input[0:(d_Vx_nearest[i+1])], axis=0)
        elif i > 0:
            small_df = size_dist_volume.iloc[(d_Vx_nearest[i]+1):(d_Vx_nearest[i+1]),:]
            V_df.iloc[:,i] = np.trapz(small_df, x=ln_size_dist_diameter_input[(d_Vx_nearest[i]+1):(d_Vx_nearest[i+1])], axis=0)

    #calculate areaXY_number and volumeXY_number
    areaXY_number = pd.Series(np.trapz(size_dist_input, x=ln_size_dist_diameter_input, axis=0))
    volumeXY_number = pd.Series(np.trapz(size_dist_volume, x=ln_size_dist_diameter_input, axis=0))

    #FIXME: question: can we replace 0 with np.nan?
    areaXY_number.replace(0, np.nan, inplace=True)
    volumeXY_number.replace(0, np.nan, inplace=True)
    print(volumeXY_number.head())

    #divide N by areaXY for f_N dataframe
    for i in range(num_bins):
        F_N_df.iloc[:,i] = N_df.iloc[:,i] / areaXY_number

    for i in range(num_bins):
        F_V_df.iloc[:,i] = V_df.iloc[:,i] / volumeXY_number

    #put in big dataframe!
    file_M_1 = pd.DataFrame() 
    file_M_1['datetime'] = datetimedf
    for i in range(num_bins):
        file_M_1['N'+str(i+1)] = N_df.iloc[:,i]
        file_M_1['F_N'+str(i+1)] = F_N_df.iloc[:,i]
        file_M_1['V'+str(i+1)] = V_df.iloc[:,i]
        file_M_1['F_V'+str(i+1)] = F_V_df.iloc[:,i]
    file_M_1['V_total'] = volumeXY_number
    file_M_1['N_total'] = areaXY_number

    filepath = 'C:/Users/cphal/OneDrive/Desktop/Aerosols/Module A/Module_A_out/'
    export_csv = file_M_1.to_csv(filepath+new_file_name+date+'.csv', index = None, header=True) 
    return(file_M_1)

In [83]:
dfDiam = createDiamDf()
file_M_1 = module_a(filename,dfDiam)

[26]
0    0.000651
1         NaN
2    0.001214
3         NaN
4    2.668080
dtype: float64


In [46]:
file_M_1.head()

,datetime,N1,F_N1,V1,F_V1,N2,F_N2,V2,F_V2,N3,F_N3,V3,F_V3,V_total,N_total
0,2016-06-18 16:59:37,0.0,0.0,0.0,0.0,0.39485,0.160365,287991952.685927,0.010181,1.538128,0.624696,27253315396.258575,0.96346,2.828691e+10,2.462202
1,2016-06-18 16:59:38,0.0,0.0,0.0,0.0,0.262086,0.075813,504086249.606054,0.027983,2.318483,0.670665,14857023467.087732,0.82475,1.801398e+10,3.456991
2,2016-06-18 16:59:39,0.0,0.0,0.0,0.0,1.837698,0.203053,1685072659.00494,0.0505,4.732053,0.522858,24246387213.59481,0.726641,3.336778e+10,9.050354
3,2016-06-18 16:59:40,0.0,0.0,0.0,0.0,3.277865,0.264623,2782028975.878111,0.051341,4.680603,0.377866,45546204897.468674,0.840528,5.418761e+10,12.386926
4,2016-06-18 16:59:41,0.0,0.0,0.0,0.0,2.355982,0.365377,1294264416.404371,0.047899,2.755922,0.427402,22429713331.354744,0.830093,2.702073e+10,6.448079


In [14]:
#MAIN METHOD
#set directory
fileDirectory = str('C:/Users/cphal/OneDrive/Desktop/Aerosols/Module A/Failure_Bin_Loc/')

dfDiam = createDiamDf()

#get filenames in folder
path = os.chdir(fileDirectory)
with os.scandir(path) as entries:
    for entry in entries:
        filename = fileDirectory + entry.name
        print('Processing ' + entry.name)
        module_a(filename, dfDiam)

In [25]:
dfDiam

,file_name,row,column_start,column_end,diameters
0,KORUS_CPSPD_,descr_long,3,32,"[0.8, 1.0, 1.2, 1.6, 2.0, 3.0, 4.0, 5.0, 6.0, ..."
1,KORUS_LAS_,descr_long,3,31,"[100.0, 112.0, 126.0, 141.0, 158.0, 178.0, 200..."
2,KORUS_SMPS_,descr_long,5,32,"[10.0, 11.2, 12.6, 14.1, 15.8, 17.8, 20.0, 22...."
3,SEAC4RS_APS_,var_name,6,20,"[562.0, 631.0, 708.0, 794.0, 891.0, 1000.0, 12..."
4,SEAC4RS_LAS_,var_name,6,35,"[100.0, 112.0, 126.0, 141.0, 158.0, 178.0, 200..."
5,SEAC4RS_SMPS_,var_name,7,37,"[11.0, 13.0, 14.0, 16.0, 18.0, 20.0, 22.0, 25...."
6,DC3_SMPS_,var_name,7,37,"[11.0, 13.0, 14.0, 16.0, 18.0, 20.0, 22.0, 25...."
7,DC3_UHSAS_,var_name,5,30,"[63.0, 71.0, 79.0, 89.0, 100.0, 112.0, 126.0, ..."
8,DISCOVER_APS_,var_name,4,17,"[562.0, 631.0, 708.0, 794.0, 891.0, 1000.0, 12..."
9,DISCOVER_CAS_,descr,7,36,"[0.68, 0.75, 0.82, 0.89, 0.96, 1.03, 1.1, 1.17..."


In [ ]:
dfDiam['diameters'][]